# Knowledge Based System by Gemini LLM

### Importing Necessary Libraries

In [131]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import getpass
import requests
from IPython.display import Image
from langchain_core.prompts import ChatPromptTemplate
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings # Import HuggingFaceEmbeddings here
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub
from huggingface_hub import login
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings('ignore')
from langchain.chains import LLMChain
load_dotenv()

False

### Setting up API Keys

In [132]:
with open("/content/drive/MyDrive/Project/Chat_with_PDF/env.txt", "r") as file:
    # Read the line and extract the key part
    line = file.read().strip()
    # Use string splitting to extract the value between the quotes
    GOOGLE_API_KEY = line.split('=')[1].strip().strip('"')

os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

### Generating poem by LLM

In [133]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
result = llm.invoke("Write a poem about Bangladesh")
print(result.content)

A land of emerald rice fields, where rivers flow,
The Ganges, Brahmaputra, a watery show.
Bangladesh, a tapestry of green and blue,
Where life unfolds, vibrant and new.

The Sundarbans, a mangrove maze,
Home to tigers, a wild, watery haze.
The Bay of Bengal, a vast, restless sea,
Where fishermen sail, with stories to be.

From Dhaka's bustling streets, to Chittagong's port,
History whispers, in every fort.
The Liberation War, a fight for freedom's call,
A nation reborn, standing tall.

The spirit of resilience, in every heart,
A tapestry of cultures, a work of art.
With laughter and song, they celebrate life,
In this land of beauty, despite the strife.

The call to prayer, a gentle chime,
Echoes across the land, transcending time.
Bangladesh, a land of hope and grace,
A vibrant soul, in this sacred space. 



### German Translator

In [134]:
system_template = "Translate the following from English into {language}"
query = """

        How are you? What do you know about Berlin?

 """
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", query)]
)

chain = prompt_template | llm
response = chain.invoke({"language": "Germany"})
print(response.content)

Wie geht es dir? Was weißt du über Berlin? 



### Question Answer query from Multiple PDF

In [135]:
def get_pdf_text(folder_path):
    text = ""

    # Search for all PDF files in the specified folder
    pdf_docs = [file for file in os.listdir(folder_path) if file.endswith('.pdf')]

    for pdf in pdf_docs:
        # Construct the full path to each PDF file
        pdf_path = os.path.join(folder_path, pdf)
        pdf_reader = PdfReader(pdf_path)

        for page in pdf_reader.pages:
            text += page.extract_text()

    return text


def get_text_chunks(text):
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )
  chunks = text_splitter.split_text(text)
  return chunks

In [139]:
path = "/content/drive/MyDrive/Project/Chat_with_PDF/Data"

text = get_pdf_text(pdf_path)
chunks = get_text_chunks(text)

system_template = """

          ### EXTRACTED TEXT FROM PDF
          {extracted_data}

          ### INSTRUCTIONS
          You are a Knowledgebased Agent. Your job is to response user query from the PDF which is stored in {extracted_data}.
          User will ask any questions reagrding the content in the PDF. You have to answer them apropriately.

          ### ONLY VALID ANSWER
          ### IF NOT IN THE PDF, JUST REPLY NOT IN THE PDF

      """


query = """
    How many person in the pdf?
 """



prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", query)]
)

chain = prompt_template | llm
response = chain.invoke({"extracted_data": chunks})
print(response.content)

There are 6 people mentioned in the PDF. 

